In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_2/models/B5R2b5_AN_1FC_fold2_3.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1032, 22)
(1032, 22)
Normal:  (559, 22)
Abnormal:  (473, 22)


,Unnamed: 0,Unnamed: 0.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,Views,...,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,filename
0,0,1,15,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,141.0,96.0,586.0,614.0,0.143898,0.107041,0.595628,0.683267,AB01 P1 C015.JPG
1,1,11,15,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,257.0,103.0,538.0,634.0,0.256617,0.115278,0.537205,0.705051,AB02 P1 C015.JPG
2,2,17,15,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,0.0,184.0,573.0,751.0,0.000862,0.190025,0.638344,0.773737,AB03 P1 C015.JPG
3,3,21,15,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,278.0,87.0,515.0,496.0,0.278395,0.097096,0.515427,0.551515,AB04 P1 C015.JPG
4,4,23,15,P1,P1,Abnormal,AB05,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,FP-A,...,https://irisprodseatraining.blob.core.windows....,249.0,69.0,439.0,385.0,0.254909,0.076894,0.448980,0.428283,AB05 P1 C015.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1032 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.03074892, -0.07016404,  0.40624246, ..., -0.08580976,
        -0.07714406, -0.14363822],
       [ 0.18978217, -0.111687  ,  0.1725129 , ..., -0.1617135 ,
        -0.06227706, -0.13980851],
       [ 0.0924086 , -0.1024469 , -0.19368407, ..., -0.08751395,
         0.0573837 , -0.1719389 ],
       ...,
       [ 0.02970364,  0.11674928, -0.10432661, ...,  0.40934062,
        -0.17891756,  0.02043705],
       [-0.17292529,  0.18678083, -0.20572427, ...,  0.05077044,
        -0.12426285,  0.01478832],
       [-0.14507236,  0.16890188, -0.02059726, ...,  0.49609345,
        -0.11752846,  0.28138596]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.030749,-0.070164,0.406242,-0.007684,-0.192478,0.014083,0.201650,-0.124938,-0.159221,-0.003187,...,-0.157478,0.065727,-0.125455,-0.072699,-0.147484,0.086054,-0.087210,-0.085810,-0.077144,-0.143638
1,0.189782,-0.111687,0.172513,-0.104830,-0.211623,-0.038904,0.077296,-0.219390,-0.182801,-0.159946,...,-0.079789,-0.118004,-0.133815,-0.169018,-0.180258,-0.142954,-0.098490,-0.161713,-0.062277,-0.139809
2,0.092409,-0.102447,-0.193684,0.006231,-0.208734,0.263452,-0.043803,-0.202135,-0.209097,-0.083969,...,-0.170690,0.038737,-0.194587,-0.171893,-0.136750,-0.106736,0.017480,-0.087514,0.057384,-0.171939
3,0.429902,-0.126829,-0.006561,0.365914,-0.225198,0.951141,0.558952,-0.209046,-0.170255,-0.042547,...,-0.166926,0.498621,-0.128891,-0.168414,-0.190584,0.596779,0.418436,-0.163215,0.349162,-0.186472
4,-0.103945,-0.144751,-0.158433,0.034403,-0.199201,-0.142411,0.000083,-0.168239,-0.159319,-0.176311,...,0.014216,-0.093562,-0.097006,-0.133427,-0.124332,0.290406,-0.117424,-0.164280,-0.149426,-0.170234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,-0.126907,0.235355,-0.170540,-0.082677,0.081396,-0.191244,-0.167280,0.050257,0.402842,-0.114634,...,0.662973,-0.163908,0.245017,0.078052,0.255321,-0.169664,-0.150968,0.173977,-0.134009,0.084714
1028,-0.125776,-0.001517,-0.086864,-0.141054,0.000828,-0.195174,-0.148154,0.028786,-0.069328,-0.167497,...,-0.025941,-0.204122,0.011646,-0.095093,-0.009956,-0.074548,-0.142799,0.299484,-0.137355,-0.025468
1029,0.029704,0.116749,-0.104327,-0.156967,0.130219,-0.141796,-0.211975,0.122765,0.052736,-0.188180,...,0.017539,-0.154426,0.230103,-0.163557,0.043775,-0.054111,-0.162472,0.409341,-0.178918,0.020437
1030,-0.172925,0.186781,-0.205724,-0.095766,0.132694,0.024460,-0.195009,0.078901,0.235401,-0.133026,...,0.700498,-0.164198,0.152492,0.199386,0.293075,-0.190331,-0.200067,0.050770,-0.124263,0.014788


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.030749,-0.070164,0.406242,-0.007684,-0.192478,0.014083,0.201650,-0.124938,-0.159221,-0.003187,...,-0.147484,0.086054,-0.087210,-0.085810,-0.077144,-0.143638,Abnormal,P1,FP-A,AB01
1,0.189782,-0.111687,0.172513,-0.104830,-0.211623,-0.038904,0.077296,-0.219390,-0.182801,-0.159946,...,-0.180258,-0.142954,-0.098490,-0.161713,-0.062277,-0.139809,Abnormal,P1,FP-A,AB02
2,0.092409,-0.102447,-0.193684,0.006231,-0.208734,0.263452,-0.043803,-0.202135,-0.209097,-0.083969,...,-0.136750,-0.106736,0.017480,-0.087514,0.057384,-0.171939,Abnormal,P1,FP-A,AB03
3,0.429902,-0.126829,-0.006561,0.365914,-0.225198,0.951141,0.558952,-0.209046,-0.170255,-0.042547,...,-0.190584,0.596779,0.418436,-0.163215,0.349162,-0.186472,Abnormal,P1,FP-A,AB04
4,-0.103945,-0.144751,-0.158433,0.034403,-0.199201,-0.142411,0.000083,-0.168239,-0.159319,-0.176311,...,-0.124332,0.290406,-0.117424,-0.164280,-0.149426,-0.170234,Abnormal,P1,FP-A,AB05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,-0.126907,0.235355,-0.170540,-0.082677,0.081396,-0.191244,-0.167280,0.050257,0.402842,-0.114634,...,0.255321,-0.169664,-0.150968,0.173977,-0.134009,0.084714,Normal,P8,FP-E,Normal
1028,-0.125776,-0.001517,-0.086864,-0.141054,0.000828,-0.195174,-0.148154,0.028786,-0.069328,-0.167497,...,-0.009956,-0.074548,-0.142799,0.299484,-0.137355,-0.025468,Normal,P8,FP-E,Normal
1029,0.029704,0.116749,-0.104327,-0.156967,0.130219,-0.141796,-0.211975,0.122765,0.052736,-0.188180,...,0.043775,-0.054111,-0.162472,0.409341,-0.178918,0.020437,Normal,P8,FP-E,Normal
1030,-0.172925,0.186781,-0.205724,-0.095766,0.132694,0.024460,-0.195009,0.078901,0.235401,-0.133026,...,0.293075,-0.190331,-0.200067,0.050770,-0.124263,0.014788,Normal,P8,FP-E,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold1_3.csv')